In [3]:
## Libraries:

# a) Bond graphs
import BondGraphTools as bgt

# b) Data extraction
from pyomexmeta import RDF, eUriType
import os
import sys
import xml.etree.ElementTree as ET
import rdflib
from lxml import etree
import re

# c) General
import copy
import difflib
import numpy as np
import pandas as pd
import math
import operator as op
import ast
import re
from termcolor import colored


# d) Plot
import matplotlib.pyplot as plt
from matplotlib import markers
import matplotlib.font_manager as font_manager
import matplotlib.colors
from pylab import rcParams
import matplotlib.image as mpimg

# sbml
from libsbml import*
import simplesbml 

# Integration and fitting
from scipy.optimize import curve_fit, minimize, least_squares, newton_krylov, basinhopping
from scipy  import integrate
import scipy.stats as stats
from symfit import parameters, variables, log, Fit, GreaterThan, Model
from symfit.core.minimizers import BasinHopping


The tests rely on tellurium to construct the models
Since tellurium is not installed the tests can't be run
If you want to run the tests, pip install tellurium first


# The code to iterate through models in a directory and count the proper models which meet the criteria for bond graph conversion. 

## Since we plan to expand the application of our tool to other reaction types and the fact that many reactions in SBML models are not annotated consistently, we didn't rule out the models which included other reactions or no SBO terms at all (Assuming that most reactions will be covered in future and some others can be approximated using mass action or Michaelis-Menten)

## Also, events are not part of biological models. We tracked the use of events in the models but we didn't rule out SBML models with events since they can be converted into bond graphs if  the events(s) is(are) simply removed (assuming that other criteria are met). 

In [16]:
# The code to count models which are potentially suitable for bond graph conversion

directory  = '/Users/nsha457/Documents/Jupyter_files/SBML_AnnotMerge/CriteriaCheck/biomodels'
# iterate over files in the directory

potentialModels=[]

# reversible MA
reversibleMaSBO = np.linspace(69,139,num=71,endpoint=True,retstep=False,dtype=int)
reversibleMaSBO=np.append(reversibleMaSBO,49)
reversibleMaSBO=np.append(reversibleMaSBO,646)

# irreversible MA
irreversibleMaSBO = np.linspace(41,61,num=21,endpoint=True,retstep=False,dtype=int)
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(140,146,num=7,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(163,166,num=4,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(560,564,num=5,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,333)

# irreversible MM  
irreversibleMMSBO = [29,187]

# reversible MM  
reversibleMMSBO = [438]


reversibleMA = {}
reversibleMM = {}
irreversibleMM = {}
irreversibleMA = {}
unknown = {}
            
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(path) and path.endswith('.xml'):
        modelName = os.path.basename(path)
        print(modelName)
        reader = SBMLReader()
        document = reader.readSBML(path)
        error = document.getNumErrors()

        if error:            
            print("Error found in ",path)

        else:  
            
            model_lib = document.getModel()
            model_simple = simplesbml.loadSBMLFile(path)
            
            
            reversibleMA[modelName]=[]; irreversibleMM[modelName]=[];
            irreversibleMA[modelName]=[]; reversibleMM[modelName]=[]; unknown[modelName]=[];
            
            # Checking the SBO terms
            
            for (reaction,r) in zip(model_simple.getListOfReactionIds(),range(len(model_lib.getListOfReactions()))):
        
                for i in reversibleMaSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):
                        reversibleMA[modelName].append(reaction)
                        
                        
                for i in irreversibleMaSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):
                        irreversibleMA[modelName].append(reaction)
                        
                
                for i in irreversibleMMSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):                   
                        irreversibleMM[modelName].append(reaction)
                        
                    
                for i in reversibleMMSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):                   
                        reversibleMM[modelName].append(reaction)
                        
                if reaction not in reversibleMA[modelName] and reaction not in irreversibleMA[modelName] and \
                reaction not in reversibleMM[modelName] and reaction not in irreversibleMM[modelName]:
                    unknown[modelName].append(reaction)
                    
            
            
            # Checking the number of events
            numEvents = model_simple.getNumEvents()
            
            
            # Checking the parameters to be constant
            nonConstantParams ={}
            nonConstantParams[modelName]=[]
            for param in model_lib.getListOfParameters().getListOfAllElements():
                if param.getConstant() == False:
                    nonConstantParams[modelName].append(param)
                    
           
            # Checking modifiers
            modifiers ={}
            modifiers[modelName]=[]
            for reaction in model_simple.getListOfReactionIds():
                if model_simple.getListOfModifiers(reaction) != []:
                    modifiers[modelName].append(model_simple.getListOfModifiers(reaction))
            

                    
            if  nonConstantParams[modelName] == [] and modifiers[modelName]==[]:
                potentialModels.append(modelName)

BIOMD0000000001.xml
BIOMD0000000002.xml
BIOMD0000000003.xml
BIOMD0000000004.xml
BIOMD0000000005.xml
BIOMD0000000006.xml
BIOMD0000000007.xml
BIOMD0000000008.xml
BIOMD0000000009.xml
BIOMD0000000010.xml
BIOMD0000000011.xml
BIOMD0000000012.xml
BIOMD0000000013.xml
BIOMD0000000014.xml
BIOMD0000000015.xml
BIOMD0000000016.xml
BIOMD0000000017.xml
BIOMD0000000018.xml
BIOMD0000000019.xml
BIOMD0000000020.xml
BIOMD0000000021.xml
BIOMD0000000022.xml
BIOMD0000000023.xml
BIOMD0000000024.xml
BIOMD0000000025.xml
BIOMD0000000026.xml
BIOMD0000000027.xml
BIOMD0000000028.xml
BIOMD0000000029.xml
BIOMD0000000030.xml
BIOMD0000000031.xml
BIOMD0000000032.xml
BIOMD0000000033.xml
BIOMD0000000034.xml
BIOMD0000000035.xml
BIOMD0000000036.xml
BIOMD0000000037.xml
BIOMD0000000038.xml
BIOMD0000000039.xml
BIOMD0000000040.xml
BIOMD0000000041.xml
BIOMD0000000042.xml
BIOMD0000000043.xml
BIOMD0000000044.xml
BIOMD0000000045.xml
BIOMD0000000046.xml
BIOMD0000000047.xml
BIOMD0000000048.xml
BIOMD0000000049.xml
BIOMD0000000050.xml


BIOMD0000000820.xml
BIOMD0000000821.xml
BIOMD0000000822.xml
BIOMD0000000823.xml
BIOMD0000000824.xml
BIOMD0000000825.xml
BIOMD0000000826.xml
BIOMD0000000827.xml
BIOMD0000000828.xml
BIOMD0000000829.xml
BIOMD0000000831.xml
BIOMD0000000832.xml
BIOMD0000000837.xml
BIOMD0000000838.xml
BIOMD0000000839.xml
BIOMD0000000840.xml
BIOMD0000000843.xml
BIOMD0000000844.xml
BIOMD0000000845.xml
BIOMD0000000846.xml
BIOMD0000000847.xml
BIOMD0000000848.xml
BIOMD0000000850.xml
BIOMD0000000851.xml
BIOMD0000000852.xml
BIOMD0000000853.xml
BIOMD0000000854.xml
BIOMD0000000855.xml
BIOMD0000000857.xml
BIOMD0000000858.xml
BIOMD0000000859.xml
BIOMD0000000860.xml
BIOMD0000000861.xml
BIOMD0000000863.xml
BIOMD0000000865.xml
BIOMD0000000867.xml
BIOMD0000000876.xml
BIOMD0000000877.xml
BIOMD0000000879.xml
BIOMD0000000880.xml
BIOMD0000000881.xml
BIOMD0000000886.xml
BIOMD0000000887.xml
BIOMD0000000888.xml
BIOMD0000000889.xml
BIOMD0000000890.xml
BIOMD0000000892.xml
BIOMD0000000893.xml
BIOMD0000000895.xml
BIOMD0000000896.xml


MODEL1204280022.xml
MODEL1204280023.xml
MODEL1204280025.xml
MODEL1204280026.xml
MODEL1204280027.xml
MODEL1204280028.xml
MODEL1204280029.xml
MODEL1204280030.xml
MODEL1204280031.xml
MODEL1204280032.xml
MODEL1204280033.xml
MODEL1204280034.xml
MODEL1204280035.xml
MODEL1204280037.xml
MODEL1204280038.xml
MODEL1204280039.xml
MODEL1207300000.xml
MODEL1208030000.xml
MODEL1208280001.xml
MODEL1209260000.xml
MODEL1210260004.xml
MODEL1302180001.xml
MODEL1302180002.xml
MODEL1303010000.xml
MODEL1303140000.xml
MODEL1304300000.xml
MODEL1305010000.xml
MODEL1308080003.xml
MODEL1310110037.xml
MODEL1401240000.xml
MODEL1403250001.xml
MODEL1403250001_url.xml
MODEL1403250002.xml
MODEL1403250002_url.xml
MODEL1405070000.xml
MODEL1409230001.xml
MODEL1409240001.xml
MODEL1409240002.xml
MODEL1409240003.xml
MODEL1410060000.xml
MODEL1410310000.xml
MODEL1411130000.xml
MODEL1411210000.xml
MODEL1412200000.xml
MODEL1502180000.xml
MODEL1502230000.xml
MODEL1502270000.xml
MODEL1503050000.xml
MODEL1503180002.xml
MODEL1503180

In [17]:
len(potentialModels)

166

In [18]:
166/1136

0.14612676056338028

# The code iterates through 3 of our annotated models and one random model in a directory. It checks if they meet the criteria for bond graph conversion or not. It will return the improper models along with the criteria which they do not meet.

## newModel0001.xml, newModel0002.xml, and newModel0003.xml meet all the criteria but  BIOMD0000000249.xml is not compatible due to the following reasons:
### Unknown reaction laws
### Non-constant parameters
### Including modifiers

In [19]:
# Check our models with added SBO terms.

directory  = '/Users/nsha457/Documents/Jupyter_files/SBML_AnnotMerge/CriteriaCheck/ourModels'
# iterate over files in the directory

potentialModels=[]

# reversible MA
reversibleMaSBO = np.linspace(69,139,num=71,endpoint=True,retstep=False,dtype=int)
reversibleMaSBO=np.append(reversibleMaSBO,49)
reversibleMaSBO=np.append(reversibleMaSBO,646)

# irreversible MA
irreversibleMaSBO = np.linspace(41,61,num=21,endpoint=True,retstep=False,dtype=int)
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(140,146,num=7,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(163,166,num=4,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,np.linspace(560,564,num=5,endpoint=True,retstep=False,dtype=int))
irreversibleMaSBO=np.append(irreversibleMaSBO,333)

# irreversible MM  
irreversibleMMSBO = [29,187]

# reversible MM  
reversibleMMSBO = [438]


reversibleMA = {}
reversibleMM = {}
irreversibleMM = {}
irreversibleMA = {}
unknown = {}
            
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(path) and path.endswith('.xml'):
        modelName = os.path.basename(path)
        reader = SBMLReader()
        document = reader.readSBML(path)
        error = document.getNumErrors()

        if error:            
            print("Error found in ",path)

        else:  
            
            model_lib = document.getModel()
            model_simple = simplesbml.loadSBMLFile(path)
            
            
            reversibleMA[modelName]=[]; irreversibleMM[modelName]=[];
            irreversibleMA[modelName]=[]; reversibleMM[modelName]=[]; unknown[modelName]=[];
            
            # Checking the SBO terms
            
            for (reaction,r) in zip(model_simple.getListOfReactionIds(),range(len(model_lib.getListOfReactions()))):
        
                for i in reversibleMaSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):
                        reversibleMA[modelName].append(reaction)
                        
                        
                for i in irreversibleMaSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):
                        irreversibleMA[modelName].append(reaction)
                        
                
                for i in irreversibleMMSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):                   
                        irreversibleMM[modelName].append(reaction)
                        
                    
                for i in reversibleMMSBO:
                    if SBase.getSBOTermID(model_lib.getListOfReactions()[r]).endswith(str('0')+str(i)):                   
                        reversibleMM[modelName].append(reaction)
                        
                if reaction not in reversibleMA[modelName] and reaction not in irreversibleMA[modelName] and \
                reaction not in reversibleMM[modelName] and reaction not in irreversibleMM[modelName]:
                    unknown[modelName].append(reaction)
                    
                    
            if unknown[modelName] != []:
                print('\n',colored('Unknown reaction law(s) for model: ','red'),colored(modelName,'red'))
                for reaction in unknown[modelName]:
                    print(reaction)
            
            
            # Checking the number of events
            numEvents = model_simple.getNumEvents()
            if numEvents!=0:
                print('\n',colored(modelName,'red'),colored(' has ','red'),colored(numEvents,'red'),colored(' events. The events need to be removed.','red'))
            
            
            # Checking the parameters to be constant
            nonConstantParams ={}
            nonConstantParams[modelName]=[]
            for param in model_lib.getListOfParameters().getListOfAllElements():
                if param.getConstant() == False:
                    nonConstantParams[modelName].append(param)
                    
            if nonConstantParams[modelName] != []:
                print('\n',colored('The following parameter(s) are not constant in:','red'),colored(modelName,'red'))
                for param in nonConstantParams[modelName]:
                    print(param)
            
            # Checking modifiers
            modifiers ={}
            modifiers[modelName]=[]
            for reaction in model_simple.getListOfReactionIds():
                if model_simple.getListOfModifiers(reaction) != []:
                    modifiers[modelName].append(model_simple.getListOfModifiers(reaction))
            
            if modifiers[modelName] != []:
                print('\n',colored('The following modifier(s) were found in:','red'),colored(modelName,'red'))
                for modif in modifiers[modelName]:
                    print(modif)
                    
            if  unknown[modelName] == [] and numEvents==0 and nonConstantParams[modelName] == [] and modifiers[modelName]==[]:
                potentialModels.append(modelName)


 Unknown reaction law(s) for model:  BIOMD0000000249.xml
r1
r2
r3
r4
r5
r6
r7
r8
r9
r10
r11
r12
r13
r14
r15
r16
r17
r18
r19
r20

 The following parameter(s) are not constant in: BIOMD0000000249.xml
<Parameter mu>
<Parameter beta_1>
<Parameter gamma_1>
<Parameter beta_2>
<Parameter gamma_2>
<Parameter sigma>
<Parameter Lambda_1>
<Parameter Lambda_2>
<Parameter I_1_frac>
<Parameter I_2_frac>
<Parameter S_frac>
<Parameter R1_frac>
<Parameter R2_frac>
<Parameter Rp_frac>

 The following modifier(s) were found in: BIOMD0000000249.xml
['N']
['I_1', 'I_1', 'I_1']
['I_2', 'I_2', 'I_2']
['I_1', 'I_1', 'I_1']
['I_2', 'I_2', 'I_2']


In [20]:
potentialModels

['newModel0001.xml', 'newModel0002.xml', 'newModel0003.xml']